In [3]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
from obspy.geodetics.base import gps2dist_azimuth,kilometer2degrees
from obspy.taup import TauPyModel
from obspy.signal.rotate import rotate_ne_rt
from matplotlib.ticker import MultipleLocator
from obspy import read, read_inventory, Stream, Trace
import numpy as np
from obspy.io.sac.sacpz import attach_paz
from obspy.signal.filter import envelope
import pandas as pd
import os,glob
import warnings
warnings.filterwarnings("ignore")
model = TauPyModel(model="iasp91")
client = Client("IRIS")
plt.rcParams['font.sans-serif']='Times New Roman'

In [4]:
DATA_DIR = '/Volumes/home/Research/DataBase/00_'
PZ_DIR  =  '/Volumes/home/Research/DataBase/00_PZs'
INFO_DIR = '/Volumes/home/Research/DataBase/Armenia'
freqmin = 0.04
freqmax = 0.125
FIG_DIR = f'/Volumes/home/Research/Progress/01_AMTG_record_plot_SKS_{freqmin}-{freqmax}'
if not os.path.isdir(FIG_DIR):
    os.mkdir(FIG_DIR)

Rphase = ['S','Sdiff', 'SS' ,'SSS' ,'SSSS' ,'SKS' ,'SKKS' ,'SKKKS' ,'ScP' ,'SP' ,'PS' ,'PcS','SKP','PKS' ]
Tphase = ['S','Sdiff','SS','SSS','SSSS','ScS']
# phaselist = set( Rphase + Tphase)
phaselist = ['SKS','SKKS']
phasecolor = {'SKS':'lightcoral', 'SKKS':'lightblue','SKKKS':'lightgreen'}
network= ['AM','TG']


exg = 2
arr_size=20
mmm = 10**-4



In [5]:
def SNRwindow(arr_t):
    signalbegin = arr_t -5
    signalend   = arr_t +25
    noiseend    = arr_t -20 
    noisebegin  = arr_t -20-60
    return signalbegin,signalend,noiseend,noisebegin
def checkday(num):
    if len(str(num)) == 1 :
        num = str(0)+str(num)
    return str(num)

In [6]:
starttime = UTCDateTime("2010-10-01")
endtime = UTCDateTime("2015-10-01")
cat = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=6,latitude =41.115,longitude=43.8036,
                        minradius=85,maxradius=140)
# print(cat.__str__(print_all=True))

14 Event(s) in Catalog:
2015-02-11T18:57:20.460000Z | -23.065,  -66.662 | 6.7 MW
2015-01-23T03:47:27.530000Z | -17.064, +168.554 | 6.8 MW
2014-11-01T18:57:22.350000Z | -19.802, -177.833 | 7.1 MW
2014-07-21T14:54:41.000000Z | -19.802, -178.400 | 6.9 MW
2014-05-04T09:15:52.240000Z | -24.734, +179.041 | 6.6 MW
2014-05-01T06:36:36.820000Z | -21.438, +170.310 | 6.6 MW
2014-04-11T20:29:13.610000Z | +11.720,  -86.014 | 6.6 MW
2014-02-07T08:40:13.500000Z | -15.076, +167.416 | 6.5 MW
2014-01-01T16:03:29.770000Z | -13.877, +167.272 | 6.5 MW
2013-11-23T07:48:32.020000Z | -17.161, -176.519 | 6.5 MW
2013-09-01T11:52:32.120000Z |  -7.539, +128.257 | 6.5 MW
2013-07-07T18:35:31.380000Z |  -4.013, +153.928 | 7.3 MW
2013-05-23T17:19:04.840000Z | -23.078, -177.213 | 7.4 MW
2013-02-09T14:16:06.960000Z |  +1.034,  -77.462 | 7.0 MW


In [16]:
Earthquake = []
MAGnitude=[]
NETWORK = []
STATION = []
DEGLIST = []
SKS_SNR_LIST = []
SKKS_SNR_LIST = []


for cata in cat[0:1]:
    eq_time = cata.origins[0].time
    print(eq_time)
    eq_lon = cata.origins[0].longitude
    eq_lat = cata.origins[0].latitude
    depth  = cata.origins[0].depth/1000
    mag    = cata.magnitudes[0].mag
    mag_type = cata.magnitudes[0].magnitude_type
    
    inv = client.get_stations(network="GO", station='*', channel="*",
                                    starttime=eq_time,endtime=eq_time+30*60)
    inv1 = client.get_stations(network="IU", station='GNI', channel="*",
                                    starttime=eq_time,endtime=eq_time+30*60)
    inv2 = client.get_stations(network="II", station='KIV', channel="*",
                                    starttime=eq_time,endtime=eq_time+30*60)
    inv = inv + inv1 + inv2   

##========================================================
    i = 0 
    for net_i in inv:
        NET=net_i.code
        if NET == 'GO':
            preflit = [0.001,0.005,45,50]
        elif NET == 'II' or NET == 'IU':
            preflit = [0.001,0.005,9,10]
        for sta_i in net_i:
            STA = sta_i.code
            st_lat = sta_i.latitude
            st_lon = sta_i.longitude
            dist,azi,baz = gps2dist_azimuth(eq_lat,eq_lon,st_lat,st_lon)
            dist = dist/1000
            deg = kilometer2degrees(dist)
            
            arrivals = model.get_travel_times(source_depth_in_km=depth,distance_in_degree=deg,phase_list=phaselist)
            arr_SKS = arrivals[0].time
            arr_SKKS = arrivals[1].time            
            
            i+=1
            try:
                ori_st = client.get_waveforms(NET, STA,'*','*',starttime=eq_time+arr_SKS-120, endtime=eq_time+arr_SKKS+100,attach_response=True)
                st = ori_st.copy()
                st.remove_response(pre_filt = preflit, output="DISP")
                st.detrend('linear')
                st.detrend('demean')
                st.taper(0.05,type='cosine')
                st.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=4,zerophase=True)
                dt = 1 / st[0].stats.sampling_rate
                if NET == 'GO':
                    HHE = st.select(component='E')[0].data
                    HHN = st.select(component='N')[0].data
                    HHR,HHT = rotate_ne_rt(HHN,HHE,baz)
                elif NET == 'IU' or NET == 'II' :
                    HH2 = st.select(location='00', channel='BH2')[0].data
                    HH1 = st.select(location='00', channel='BH1')[0].data
                    HHR,HHT = rotate_ne_rt(HH1,HH2,baz)

                Earthquake.append(eq_time)
                MAGnitude.append(mag)
                NETWORK.append(NET)
                STATION.append(STA)
                DEGLIST.append(deg)
    ##==========================calculate SNR =========================       
                SKS_signalbegin,SKS_signalend,SKS_noiseend,SKS_noisebegin = SNRwindow(arr_SKS)
                temp_tr = Trace(data=HHR)
                temp_tr.stats.delta = dt
                temp_tr.stats.starttime = st[0].stats.starttime

                SKS_noise = temp_tr.slice(starttime=eq_time+SKS_noisebegin,endtime=eq_time+SKS_noiseend)
                SKS_signal = temp_tr.slice(starttime=eq_time+SKS_signalbegin ,endtime = eq_time+SKS_signalend)
                SKS_signal_envelope = envelope(SKS_signal.data)
                SKS_noise_envelope = envelope(SKS_noise.data)

                SKS_cal_signal = sum(SKS_signal_envelope**2)
                SKS_cal_noise = sum(SKS_noise_envelope**2)

                SKS_SNR = int(SKS_cal_signal * 2 / SKS_cal_noise)
                if SKS_signalbegin < arr_SKKS < SKS_signalend: 
#                     print('SKKS nan')
                    SKKS_SNR = np.nan
                else : 
                    SKKS_signalbegin,SKKS_signalend,SKKS_noiseend,SKKS_noisebegin = SNRwindow(arr_SKKS)
                    SKKS_signal = temp_tr.slice(starttime=eq_time+SKKS_signalbegin ,endtime = eq_time+SKKS_signalend)
                    SKKS_signal_envelope = envelope(SKKS_signal.data)
                    SKKS_cal_signal = sum(SKKS_signal_envelope**2)
                    SKKS_SNR = int(SKKS_cal_signal * 2 / SKS_cal_noise)
                SKS_SNR_LIST.append(SKS_SNR)
                SKKS_SNR_LIST.append(SKKS_SNR)
            except: pass
#                 print(f'{NET} {STA} byebye') 
#                 SKS_SNR = np.nan
#                 SKKS_SNR = np.nan


######==============變數刪除================    
        del locals()['arr_SKS']
        del locals()['arr_SKKS']  

    
SNRdf  = pd.DataFrame({'UTCDateTime':Earthquake,
                       'Magnitude':MAGnitude,
                      'Network':NETWORK,
                      'Station':STATION,
                      'Dist':DEGLIST,
                      'SKS_SNR':SKS_SNR_LIST,
                      'SKKS_SNR':SKKS_SNR_LIST})  


SNRdf.to_csv(f'/Volumes/home/Research/Progress/01_GO_phase_SNR_{freqmin}-{freqmax}.csv',index=False)

2015-02-11T18:57:20.460000Z
GO AKH byebye
GO BATM byebye
GO BGD byebye
GO CHVG byebye
GO DDFL byebye
GO DGRG byebye
GO GUDG byebye
GO KZRT byebye
GO LGD byebye
GO SEAG byebye
GO TBLG byebye
GO TRLG byebye
